In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# ---------------- CREATE DATASET ----------------

In [4]:
# constants
block_size = 3

In [3]:
# get words
words = open('names.txt', 'r').read().splitlines()
words[:5]

['emma', 'olivia', 'ava', 'isabella', 'sophia']

In [10]:
# create LUTs
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [18]:
# build dataset
def build_dataset(words):
    
    X, Y = [], []

    for w in words:

        # starting context is all 0s
        context = [0] * block_size

        # add end char and iterate through word
        for ch in w + '.':
            
            # convert char to index
            ix = stoi[ch]

            # update dataset
            X.append(context)
            Y.append(ix)

            # shift context
            context = context[1:] + [ix]

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

In [34]:
# create splits
import random
random.seed(42)
random.shuffle(words)

n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xtest, Ytest = build_dataset(words[n2:])

torch.Size([182771, 3]) torch.Size([182771])
torch.Size([22711, 3]) torch.Size([22711])
torch.Size([22664, 3]) torch.Size([22664])


In [25]:
# ---------------- IMPLEMENT MLP ----------------

In [36]:
vocab_size = 27
n_embd = 10
n_hidden = 200

In [38]:
g = torch.Generator().manual_seed(2147483647)
# char embedding vectors
C = torch.randn((vocab_size, n_embd),             generator=g)

# weights and biases
W1 = torch.randn((block_size * n_embd, n_hidden), generator=g)
b1 = torch.randn(n_hidden,                        generator=g)
W2 = torch.randn((n_hidden, vocab_size),         generator=g)
b2 = torch.randn(vocab_size,                      generator=g)

parameters = [C, W1, b1, W2, b2]
for p in parameters:
    p.requires_grad = True

In [40]:
g = torch.Generator().manual_seed(2147483647)
# train
batch_size = 32
max_steps = 200000

for i in range(max_steps):

    # minibatch construct
    ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
    Xb, Yb = Xtr[ix], Ytr[ix]

    # forward pass
    emb = C[Xb]
    h = torch.tanh(emb.view(emb.shape[0], -1) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Yb)

    if i % 10000 == 0:
        print(loss)

    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()

    # update
    lr = 0.1 if i < 100000 else 0.01
    for p in parameters:
        p.data += -lr * p.grad

tensor(25.4327, grad_fn=<NllLossBackward0>)
tensor(2.6777, grad_fn=<NllLossBackward0>)
tensor(2.5408, grad_fn=<NllLossBackward0>)
tensor(2.3156, grad_fn=<NllLossBackward0>)
tensor(2.1878, grad_fn=<NllLossBackward0>)
tensor(2.3344, grad_fn=<NllLossBackward0>)
tensor(2.4377, grad_fn=<NllLossBackward0>)
tensor(2.4165, grad_fn=<NllLossBackward0>)
tensor(2.1740, grad_fn=<NllLossBackward0>)
tensor(2.0551, grad_fn=<NllLossBackward0>)
tensor(2.3760, grad_fn=<NllLossBackward0>)
tensor(2.1351, grad_fn=<NllLossBackward0>)
tensor(2.4657, grad_fn=<NllLossBackward0>)
tensor(2.0169, grad_fn=<NllLossBackward0>)
tensor(1.9586, grad_fn=<NllLossBackward0>)
tensor(2.4092, grad_fn=<NllLossBackward0>)
tensor(2.1924, grad_fn=<NllLossBackward0>)
tensor(2.0844, grad_fn=<NllLossBackward0>)
tensor(2.3244, grad_fn=<NllLossBackward0>)
tensor(2.3881, grad_fn=<NllLossBackward0>)


In [ ]:
# ---------------- SAMPLE FROM MLP ----------------

In [44]:
for _ in range(10):

    out = []
    context = [0] * block_size

    while True:
        emb = C[torch.tensor(context)]
        h = torch.tanh(emb.view(1, -1) @ W1 + b1)
        logits = h @ W2 + b2
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples=1).item()

        out.append(ix)

        context = context[1:] + [ix]
        if ix == 0:
            break

    print(''.join(itos[i] for i in out))

rilynn.
kna.
marisleei.
sylanya.
sah.
nerahiod.
beviydtauguna.
jmaja.
skylori.
card.
